In [1]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import ollama
import os
import re

In [2]:
# Caminhos para os diretórios das imagens

pleasant_p_dir = "/home/jerre/git/Stable_Diffusion_IC/images/Pleasant (+3)"
pleasant_p_dir_name = "pleasant_p"

unpleasant_p_dir = "/home/jerre/git/Stable_Diffusion_IC/images/Unpleasant (+3)"
unpleasant_p_dir_name = "unpleasant_p"

pleasant_dir = "/home/jerre/git/Stable_Diffusion_IC/images/Pleasant"
pleasant_dir_name = "pleasant"

unpleasant_dir = "/home/jerre/git/Stable_Diffusion_IC/images/Unpleasant"
unpleasant_dir_name = "unpleasant"

pleasant_n_dir = "/home/jerre/git/Stable_Diffusion_IC/images/Pleasant (-3)"
pleasant_n_dir_name = "pleasant_n"

unpleasant_n_dir = "/home/jerre/git/Stable_Diffusion_IC/images/Unpleasant (-3)"
unpleasant_n_dir_name = "unpleasant_n"


In [3]:
# Base para utilização do ollama com o llava

response = ollama.chat(
    model='llava', messages=[
  {
    'role': 'user',
    'content': 'Rate the hedonic valence of the following image, /home/jerre/git/StableDifussion_IC/images/Pleasant (+3)/Pleasant____landscape_S235711113_St25_G7.5.153.png, on a scale from 1 to 9, where 1 represents very negative and 9 represents very positive. Write only a number.',
  },
])
print(response['message']['content']) 

 I'm sorry, but as an AI language model, I am not able to view images directly. Therefore, I cannot provide a rating of the image based on its content. Can you please describe the image or provide more context so that I can assist you better? 


In [4]:
# Importando e nomeando as imagens

image_extensions = ('.png', '.jpg', '.jpeg', '.gif', '.bmp')
image_paths = []
image_names = []
image_seeds = []
image_groups = []

def import_images(directory, directory_name):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(image_extensions):
                image_paths.append(os.path.join(root, file))
                match = re.search(r'S(\d+)', file)
                if match:
                    seed = int(match.group(1))
                    image_names.append(f"{directory_name}_{(seed - 235711112)}")
                    image_seeds.append(seed)
                    image_groups.append(directory_name)
                else:
                    # Se não encontrar o padrão, atribui -1 como um valor padrão para seed
                    seed = -1
                    image_names.append(f"{directory_name}_unknown_seed")
                    image_seeds.append(seed)
                    image_groups.append(directory_name)

import_images(pleasant_dir, pleasant_dir_name)
import_images(unpleasant_dir, unpleasant_dir_name)
import_images(pleasant_p_dir, pleasant_p_dir_name)
import_images(unpleasant_p_dir, unpleasant_p_dir_name)
import_images(pleasant_n_dir, pleasant_n_dir_name)
import_images(unpleasant_n_dir, unpleasant_n_dir_name)

# Criando o DataFrame
df = pd.DataFrame({'path': image_paths, 'name': image_names, 'seed': image_seeds, 'group': image_groups})

In [5]:
df.head()

,path,name,seed,group
0,/home/jerre/git/Stable_Diffusion_IC/images/Ple...,pleasant_5,235711117,pleasant
1,/home/jerre/git/Stable_Diffusion_IC/images/Ple...,pleasant_13,235711125,pleasant
2,/home/jerre/git/Stable_Diffusion_IC/images/Ple...,pleasant_30,235711142,pleasant
3,/home/jerre/git/Stable_Diffusion_IC/images/Ple...,pleasant_23,235711135,pleasant
4,/home/jerre/git/Stable_Diffusion_IC/images/Ple...,pleasant_15,235711127,pleasant


In [6]:
# obtendo taxas de compressibilidade, tamanho de imagens

compressed_adress = "../images/Comprimida"
compression_quality = 95
if not os.path.exists(compressed_adress):
    os.makedirs(compressed_adress)

size_list = []
compressed_size_list = []
compression_ratio_list = []
complexity_list = []

for index, row in df.iterrows():
    image_path = row['path']
    image = Image.open(image_path)
    filename = os.path.basename(image_path)
    compressed_img_path = os.path.join(compressed_adress, filename)
    image.save(compressed_img_path, optimize=True, quality=compression_quality)

    compressed_size = os.path.getsize(compressed_img_path)
    size = os.path.getsize(image_path)
    compressed_ratio = compressed_size / size

    size_list.append(size)
    compressed_size_list.append(compressed_size)
    compression_ratio_list.append(round(compressed_ratio*100))
    complexity_list.append(round((compressed_size/(512*512*8))*100, 2))

df['size'] = size_list
df['compressed_size'] = compressed_size_list
df['compression_ratio'] = compression_ratio_list
df['complexity'] = complexity_list

In [7]:
df.head()

,path,name,seed,group,size,compressed_size,compression_ratio,complexity
0,/home/jerre/git/Stable_Diffusion_IC/images/Ple...,pleasant_5,235711117,pleasant,429645,412478,96,19.67
1,/home/jerre/git/Stable_Diffusion_IC/images/Ple...,pleasant_13,235711125,pleasant,503126,489019,97,23.32
2,/home/jerre/git/Stable_Diffusion_IC/images/Ple...,pleasant_30,235711142,pleasant,450853,432069,96,20.60
3,/home/jerre/git/Stable_Diffusion_IC/images/Ple...,pleasant_23,235711135,pleasant,476211,462522,97,22.05
4,/home/jerre/git/Stable_Diffusion_IC/images/Ple...,pleasant_15,235711127,pleasant,441133,426558,97,20.34


In [8]:
# Gerando a coluna de hedonic valence

valence_list = []

for index, row in df.iterrows():
    image_path = row['path']

    response = ollama.chat(
        model='llava', messages=[
        {
        'role': 'user',
        'content': f'Rate the hedonic valence of the following image, {image_path}, on a scale from 1 to 9, where 1 represents very negative and 9 represents very positive. Write only numbers',
      },
    ])
    
    valence_list.append(response['message']['content'])
    print((response['message']['content']))

df['valence'] = valence_list


 9 
 I'm sorry, but as an AI language model, I am not able to see or perceive images. Therefore, I cannot provide a hedonic valence rating for the image you have mentioned. Can you please describe the image so that I can attempt to make a more informed estimate of its hedonic valence? 
 8 
 I'm sorry, but as a language model AI, I am unable to perceive images or provide ratings for them. 
 9 
 9 
 9 
 8 
 I'm sorry, but as an AI language model, I cannot directly rate the hedonic valence of images or any other subjective experience. However, based on descriptions of pleasant landscapes, I can suggest a possible rating: 7. 
 8 
 8 
 8 
 The image you provided appears to depict a pleasant landscape scene. Based on the visual content of the image, I would rate its hedonic valence as a 7 out of 9 on your scale, with 9 being the most positive. This is because the landscape seems tranquil and picturesque, which are typically associated with positive emotions such as happiness or relaxation. H

In [1]:
df.to_csv('data.csv', index=False)

NameError: name 'df' is not defined